## Please run all the cells below by going to Cell>Run All

In [ ]:
# create a folder named 'data' inside the folder the notebook file
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
path ='C:\Working\Github\Dataset\Application'

application_train=pd.read_csv(path+'/application_data.csv')
pd.options.display.max_columns =None

# Current Application

# Understanding data, counts, and find null values

In [ ]:
application_train.head()

In [ ]:
application_train.info(verbose=True,null_counts=True)

**The info and head of the data frame, it looks like there is not type coversion required and all seems OK

In [ ]:
print('shape ',application_train.shape)
print('Total records is {0}, and total columns is {1}'.format(application_train.shape[0],application_train.shape[1]))
print('Total float type columns ={0} \ntotal integer type columns ={1} \nTotal object type columns ={2}'.format(application_train.dtypes.value_counts()[0],application_train.dtypes.value_counts()[1],application_train.dtypes.value_counts()[2]))

with 106 numeric and 16 categorial variables

## Computing Class Imbalance using Target variable

In [ ]:
print('Percentage loan applications that were paid back ', round(100*sum(application_train.TARGET==0)/len(application_train),2),'%')
print('Percentage loan applications that were not paid back ', round(100*sum(application_train.TARGET==1)/len(application_train),2),'%')

In [ ]:
# Calculate the percentage of 1s over 0s to see ratio of 1s to 0s
imbalance=100*application_train['TARGET'].value_counts()/application_train.shape[0]
#Print imbalance
print(imbalance[1])

In [ ]:
sns.countplot(application_train.TARGET)
plt.xlabel('TARGET')
plt.ylabel("# of APPLICATIONS")
plt.title('DISTRIBUTION OF defaulters (1) vs non-defaults (0)')
plt.show();

##### The data is  highly imbalanced as ```1s``` compared to ```0s``` which is only about 9% of ```0s```.
##### There are more than 250000 applications able to repay the loan but there are about 2500 applications with defaults

# Data Cleaning

### 1. Information about the data, nulls,uniques

In [ ]:
pd.set_option('display.max_rows', application_train.shape[0]+1)

#calcualte nulls, percent of nulls,data type,unique,total records, etc
application_train2=application_train.agg(['count','nunique']).T
application_train2['nulls']=application_train.isnull().sum()
application_train2['percent']=(100*application_train.isnull().sum())/len(application_train)
application_train2['datatype']=application_train.dtypes

In [ ]:
application_train2.sort_values(by='percent',ascending=False)

### 1. Excluding columns with null values

In [ ]:
sum(application_train2['percent']>=14)/application_train2.shape[0]*100

   1. Out of **122** columns, **42** percent of columns have 14 % or more nulls
   2. All columns with 14% or more null values excluded in the following data frame

### 2. Treating columns having null values between 0 and 14%

In [ ]:
df_less_than_14=application_train2[application_train2.percent<14]
# sort descending the percentage column to find columns with highest and lowest null values
df_less_than_14[df_less_than_14.sort_values(by='percent', ascending=False)['percent']>0 ]


1. AMT_ANNUITY<br>
We will remove null values because 12 nulls is not much data and will not affect the overall accuracy of data
2. AMT_GOODS_PRICE<br>
Impute this column with median values

3. NAME_TYPE_SUITE<br>
Imputer the value to unknown although not much but it is a data quality issues

4. CNT_FAM_MEMBERS<br>
Impute this with mode value or remove the nulls

### Impute the following as unknow since the phone is not known
1. DAYS_LAST_PHONE_CHANGE 

#### remove nulls for the following since the nulls are very few 
1. OBS_30_CNT_SOCIAL_CIRCLE
2. DEF_30_CNT_SOCIAL_CIRCLE
3. OBS_60_CNT_SOCIAL_CIRCLE
4. DEF_60_CNT_SOCIAL_CIRCLE
5. DAYS_LAST_PHONE_CHANGE

#### Looks like categorial variable and we Impute the missing value of the following with 0 to indicate no enquiries 
1. AMT_REQ_CREDIT_BUREAU_HOUR
2. AMT_REQ_CREDIT_BUREAU_DAY
3. AMT_REQ_CREDIT_BUREAU_WEEK
4. AMT_REQ_CREDIT_BUREAU_MON
5. AMT_REQ_CREDIT_BUREAU_QRT
6. AMT_REQ_CREDIT_BUREAU_YEAR


### 3.  Columns to keep for analysis
keep all columns having less than 13% missing values and also keep OCCUPATION_TYPE since it had 31% missing values I am interested to keep this column

In [ ]:
columnstokeep=list(application_train2[application_train2.percent<14].index)
print(columnstokeep)
print(len(columnstokeep))
columnstokeep.append('OCCUPATION_TYPE')
#totally 65 columns are extracted but will do the analysis 

In [ ]:
#exclude columsn starting with FLAG_DOCUMENT_
colltokeep=[]
for i in columnstokeep:
    if i[0:14]!='FLAG_DOCUMENT_':
        colltokeep.append(i)       
        
print(colltokeep)

In [ ]:
finall_application_data=application_train[colltokeep]

In [ ]:
# now we have 52 column to go
finall_application_data.shape

#### Listing columns by object type
#### 1. Float

In [ ]:
print('Total columns ={0} \ncolumns = {1}'.format(len(finall_application_data.select_dtypes('float64').columns),finall_application_data.select_dtypes('float64').columns))

#### 2. Object

In [ ]:
print('Total columns ={0} \ncolumns = {1}'.format(len(finall_application_data.select_dtypes('object').columns),finall_application_data.select_dtypes('object').columns))

#### Int64

In [ ]:
print('Total columns ={0} \ncolumns = {1}'.format(len(finall_application_data.select_dtypes('int64').columns),finall_application_data.select_dtypes('int64').columns))

  Now there are 12 object and 40 numeric types

### Separate defaulters and non-defaulters for analysis

In [ ]:
defaulters_data=finall_application_data[finall_application_data.TARGET==1]
print(len(defaulters_data))
non_defaulters_data=finall_application_data[finall_application_data.TARGET==0]
print(len(non_defaulters_data))

# Analysis

### 1. Univariate analysis for categorial variables

In [ ]:
#custom function for creating barplot
def Target_univariate_analysis_bar(x,rotate_label=False):
    fig, (ax1, ax2) = plt.subplots(nrows=1,ncols=2, figsize=(14,5),dpi=1200)
    
    sns1=sns.countplot(ax=ax1,x=non_defaulters_data[x], data=non_defaulters_data, order= non_defaulters_data[x].value_counts().index)
    ax1.set_xlabel('%s' %x)
    ax1.set_ylabel("Number of applications")
    ax1.set_title('Distribution of '+ '%s' %x +' for Non-Defaulters',  fontsize=12,fontweight=10)
    if(rotate_label):
        sns1.set_xticklabels(sns1.get_xticklabels(),rotation=90)
        
    sns2=sns.countplot(ax=ax2,x=defaulters_data[x], data=defaulters_data, order= defaulters_data[x].value_counts().index)
    ax2.set_title('Distribution of '+ '%s' %x +' for Defaulters', fontsize=12,fontweight=10)
    ax2.set_xlabel('%s' %x)
    ax2.set_ylabel("# of applications")
    if(rotate_label):
        sns2.set_xticklabels(sns2.get_xticklabels(),rotation=90)
        
    # remove spines from the plots
    sns.despine(left=True, bottom=True, right=True)
    plt.show()

In [ ]:
#custom function for creating pie plot
def Target_univariate_analysis_pie(x):
    fig, (ax1, ax2) = plt.subplots(nrows=1,ncols=2, figsize=(15,5),dpi=1200)
    def func(pct, allvals):
        absolute = int(pct/100.*np.sum(allvals))
        return "{:.1f}%".format(pct)

    # Chart for non-defaulters
    data=list(non_defaulters_data[x].value_counts())    
    labels=list(non_defaulters_data[x].value_counts().index)
    
    wedges, texts, autotexts = ax1.pie(data, autopct=lambda pct: func(pct, data),
                                      textprops=dict(color="black"),pctdistance=0.6)
    ax1.legend(wedges, labels,
              title=x,
              loc="upper left",
              bbox_to_anchor=(1, 0, 2, 0.5))
    plt.setp(autotexts, size=11, weight="bold")
    ax1.set_title('Distribution of '+ '%s' %x +' for \nNon-Defaulters',fontsize=10,color='black')
    ax1.axis('equal')
    
    # Chart for defaulters
    data_defaulters=list(defaulters_data[x].value_counts())    
    labels_defaulters=list(defaulters_data[x].value_counts().index)
    
    wedges, texts, autotexts = ax2.pie(data_defaulters, autopct=lambda pct: func(pct, data),
                                      textprops=dict(color="black"))
    ax2.legend(wedges, labels,
          title=x,
          loc="upper left",
          bbox_to_anchor=(1, 0, 3, 0.5))
    
    plt.setp(autotexts, size=11, weight="bold")
    ax2.set_title('Distribution of '+ '%s' %x +' for \nDefaulters',fontsize=10,color='black')
    ax2.axis('equal')
    
    plt.show()

In [ ]:
#custom function for creating pie plot
def target_univariate_anlysis_pie2(x,fontcolor,colors=[]):
    plt.figure(figsize=(10,8),dpi=1200)
    data_non_defaulters=list(non_defaulters_data[x].value_counts())    
    labels_non_defaulters=list(non_defaulters_data[x].value_counts().index)
    ax1 = plt.subplot2grid((2,2),(0,0))

    plt.pie(data_non_defaulters,labels=labels_non_defaulters,autopct='%1.1f%%', shadow=False,textprops={'fontsize':8,'color':fontcolor},colors=colors)
    plt.axis('equal')
    plt.title('Distribution of '+ '%s' %x +' for \nNon-Defaulters',fontsize=10,color='black')

    data_defaulters=list(defaulters_data[x].value_counts())    
    labels_defaulters=list(defaulters_data[x].value_counts().index)
    ax2 = plt.subplot2grid((2,2),(0,1))
    plt.pie(data_defaulters,labels=labels_defaulters,autopct='%1.1f%%', shadow=False,textprops={'fontsize':8,'color':fontcolor},colors=colors)
    plt.title('Distribution of '+ '%s' %x +' for \nDefaulters',fontsize=10,color='black')
    
    plt.show()

#### 1. NAME_CONTRACT_TYPE
**Analysis:**<br>
Here, the Cash Loans are far much higher than Revolving Loans for both defaulters and non-defaulters<br>

In [ ]:
Target_univariate_analysis_bar('NAME_CONTRACT_TYPE')

In [ ]:
target_univariate_anlysis_pie2('NAME_CONTRACT_TYPE','black',['green','orange'])

#### 2. CODE_GENDER
**Analysis:**<br>
Here, Female applicants are receiving more loan than male applicants. While in percentage of male defaulter applicants are more than percentage of male in non-defaulters.<br> See following pie chart

In [ ]:
# Analysis of CODE_GENDER
Target_univariate_analysis_bar('CODE_GENDER')
sum(non_defaulters_data['CODE_GENDER']=='XNA')

In [ ]:
Target_univariate_analysis_pie('CODE_GENDER')

#### 3. FLAG_OWN_CAR
**Analysis:**<br>
Here, the proporation of car holders are similar in both defaulter and non-defaulters applications however, the number of applications owning a car is less than non-defaulters

In [ ]:
Target_univariate_analysis_pie('FLAG_OWN_CAR')

#### 4. FLAG_OWN_REALTY
**Analysis:**<br>
Here, the proporation of reallity owners are similar in both defaulter and non-defaulters applications

In [ ]:
# Analysis of FLAG_OWN_REALTY
Target_univariate_analysis_bar('FLAG_OWN_REALTY')

In [ ]:
Target_univariate_analysis_pie('FLAG_OWN_REALTY')

#### 5. NAME_HOUSING_TYPE
**Analysis:**<br>
Here, the percentage of housing type with **House/department** in defaulters is less than non-defaulters which makes over 85% of all application in both cases

In [ ]:
# Analysis of NAME_TYPE_SUITE
Target_univariate_analysis_bar('NAME_HOUSING_TYPE',True)

In [ ]:
# Analysis of NAME_TYPE_SUITE
target_univariate_anlysis_pie2('NAME_HOUSING_TYPE','black',['lightblue', 'orange', 'yellow', 'blue','violet', 'grey'])

#### 6. OCCUPATION_TYPE
**Analysis:**<br>
Here, the **laborers**, **sales staff** percentage is more in defaulters than non-defaulters. Conversly, the precetage of **Core Staff**, and **Managers** are more in non-defaulters than defaulters.

In [ ]:
# Analysis of OCCUPATION_TYPE
Target_univariate_analysis_bar('OCCUPATION_TYPE',True)

In [ ]:
# Analysis of OCCUPATION_TYPE
Target_univariate_analysis_pie('OCCUPATION_TYPE')

#### 7. NAME_FAMILY_STATUS
**Analysis:**<br>
Here, the **single/**,and  **civil mariage** percentage is more in defaulters than non-defaulters. Conversly, the precetage of **Married** is more in non-defaulters than defaulters.

In [ ]:
# Analysis of NAME_FAMILY_STATUS
Target_univariate_analysis_pie('NAME_FAMILY_STATUS')

#### 8. NAME_EDUCATION_TYPE
**Analysis:**<br>
Here, the **Seconday/secodary special** percentage is more in defaulters than non-defaulters. But, the precetage of **Higher education** holders is more in non-defaulters than defaulters.

In [ ]:
# Analysis of NAME_EDUCATION_TYPE
Target_univariate_analysis_pie('NAME_EDUCATION_TYPE')

## 2. Numeric Data Analysis<br>
List of numeric column types

In [ ]:
print('Total columns ={0} \ncolumns = {1}'.format(len(finall_application_data.select_dtypes('float64').columns),finall_application_data.select_dtypes('float64').columns))
print('Total columns ={0} \ncolumns = {1}'.format(len(finall_application_data.select_dtypes('int64').columns),finall_application_data.select_dtypes('int64').columns))

will analyse the following numeric columns<br>
AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE','DAYS_EMPLOYED

#### Create common function for ploting

In [ ]:
def numeric_data_plot(x,rotate_label=False):
    fig, (ax1, ax2) = plt.subplots(nrows=1,ncols=2, figsize=(14,5),dpi=1200)
    
    sns1=sns.boxplot(ax=ax1,y=non_defaulters_data[x], palette='muted')
    ax1.set_xlabel('%s' %x)
    ax1.set_ylabel("# of applications")
    ax1.set_title('Distribution of '+ '%s' %x +' for Non-Defaulters',  fontsize=12,fontweight=10)
    if(rotate_label):
        sns1.set_xticklabels(sns1.get_xticklabels(),rotation=90)
        
    sns2=sns.boxplot(ax=ax2,y=defaulters_data[x],palette='muted')
    ax2.set_title('Distribution of '+ '%s' %x +' for Defaulters', fontsize=12,fontweight=10)
    ax2.set_xlabel('%s' %x)
    ax2.set_ylabel("# of applications")
    if(rotate_label):
        sns2.set_xticklabels(sns2.get_xticklabels(),rotation=90)
        
    # remove spines from the plots
    sns.despine(left=True, bottom=True, right=True)
    
    plt.show()

**Plot** the column **AMT_INCOME_TOTAL** to find if there is any outlier

In [ ]:
numeric_data_plot('AMT_INCOME_TOTAL')

In [ ]:
# Check outlier status
pd.set_option('display.float_format', lambda x: '%.1f' % x)
application_train_data=finall_application_data
application_train_data.AMT_INCOME_TOTAL.describe(percentiles=[0.05,0.10,0.25,0.50,0.75,0.80,0.90,0.95,0.99,0.9999])

In [ ]:
application_train_data.shape

We clearly see there is outlier at the percentale of 95%% so we will remove it from analysis

In [ ]:
# Removing all entries above 95 percentile
application_train_data=application_train_data[application_train_data.AMT_INCOME_TOTAL<np.nanpercentile(application_train_data['AMT_INCOME_TOTAL'], 95)]

In [ ]:
# update the dataframes
defaulters_data=application_train_data[application_train_data.TARGET==1] 
non_defaulters_data=application_train_data[application_train_data.TARGET==0]

In [ ]:
def custom_histogram(x):
    plt.figure(figsize=(15,5))
    ax1=plt.subplot(1,2,1)
    plt.hist(non_defaulters_data[x],edgecolor='black')
    ax1.set_title('Distribution of '+ '%s' %x +' for Non-Defaulters', fontsize=12,fontweight=10)
    ax2=plt.subplot(1,2,2)
    plt.hist(defaulters_data[x],edgecolor='black')
    ax2.set_title('Distribution of '+ '%s' %x +' for Defaulters', fontsize=12,fontweight=10)

In [ ]:
numeric_data_plot('AMT_INCOME_TOTAL')

In [ ]:
custom_histogram('AMT_INCOME_TOTAL')

**Analysis:** <br> 

In both non-defaulters and defaulters, majority of loans are given between 90000 and 200000. <br>
The majority of loans in both defaulters and non-defaulters between 50th and 75th percentile 


#### 2. AMT_CREDIT


In [ ]:
numeric_data_plot('AMT_CREDIT')

Clearly there is outlier in the amount credited for applications <br>
**Following, we need to treat view and treet outliers**

In [ ]:
# view outlier status
pd.set_option('display.float_format', lambda x: '%.1f' % x)
application_train_data.AMT_CREDIT.describe(percentiles=[0.05,0.10,0.25,0.50,0.75,0.80,0.90,0.95,0.99,0.9999])

In [ ]:
# Removing all entries above 99.99 percentile
application_train_data=application_train_data[application_train_data.AMT_CREDIT<np.nanpercentile(application_train_data['AMT_CREDIT'],99)]

# update the dataframes
defaulters_data=application_train_data[application_train_data.TARGET==1] 
non_defaulters_data=application_train_data[application_train_data.TARGET==0]

numeric_data_plot('AMT_CREDIT')


In [ ]:
custom_histogram('AMT_CREDIT')

**Analysis:** <br> 
We see that the CREDIT amount lies in the range of 250000 and 750000 for non defaulters and defaulter ranges between 200000 to 800000

#### 3. AMT_ANNUITY


In [ ]:
numeric_data_plot('AMT_ANNUITY')

In [ ]:
# Removing all entries above 99 percentile
application_train_data=application_train_data[application_train_data.AMT_ANNUITY<np.nanpercentile(application_train_data['AMT_ANNUITY'],99)]

# update the dataframes
defaulters_data=application_train_data[application_train_data.TARGET==1] 
non_defaulters_data=application_train_data[application_train_data.TARGET==0]

numeric_data_plot('AMT_ANNUITY')

In [ ]:
custom_histogram('AMT_ANNUITY')

**Analysis:** <br> 
No special differences between defaulters and non-defaulters in AMT_ANNUITY

## Binning contineous variables

#### 1. AMT_INCOME_TOTAL

In [ ]:
application_train_data['AMT_INCOME_TOTAL'].describe()

In [ ]:
# Creating bins of income bracket and specifying ceratin labels to them using pd.qcut

defaulters_data['AMT_INCOME_TOTAL_BINS'] = pd.qcut(application_train_data['AMT_INCOME_TOTAL'],q=[0,0.10,0.35,0.50,0.90,1], labels=['Lowest','Low','Medium','High','Highest']).copy()
non_defaulters_data['AMT_INCOME_TOTAL_BINS'] = pd.qcut(application_train_data['AMT_INCOME_TOTAL'],q=[0,0.10,0.35,0.50,0.90,1], labels=['Lowest','Low','Medium','High','Highest']).copy()


In [ ]:
Target_univariate_analysis_bar('AMT_INCOME_TOTAL_BINS')

In [ ]:
target_univariate_anlysis_pie2('AMT_INCOME_TOTAL_BINS','black',['blue','orange','green','grey','purple'])

**Analysis:**<br>
No clear difference between defaulters and non-defaulters**

#### 2. AMT_ANNUITY

In [ ]:
# Creating bins of income bracket and specifying ceratin labels to them using pd.qcut

defaulters_data['AMT_ANNUITY_BINS'] = pd.qcut(application_train_data['AMT_ANNUITY'],q=[0,0.10,0.35,0.50,0.90,1], labels=['Lowest','Low','Medium','High','Highest']).copy()
non_defaulters_data['AMT_ANNUITY_BINS'] = pd.qcut(application_train_data['AMT_ANNUITY'],q=[0,0.10,0.35,0.50,0.90,1], labels=['Lowest','Low','Medium','High','Highest']).copy()


In [ ]:
Target_univariate_analysis_bar('AMT_ANNUITY_BINS')

In [ ]:
target_univariate_anlysis_pie2('AMT_ANNUITY_BINS','black',['blue','orange','green','grey','purple'])

**Analysis:**<br>
About **44%** of defaulters have high annuity value that lies between 50th to 90th percentile

## Bi-Variate Analysis

In [ ]:
non_defaulters_data.select_dtypes('int64').columns

In [ ]:
non_defaulters_data.select_dtypes('float64').columns

In [ ]:
# Columsn to do fo analysis

bivar_cols=['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'CNT_FAM_MEMBERS',
       'EXT_SOURCE_2', 'CNT_CHILDREN','REGION_RATING_CLIENT',
       'DAYS_LAST_PHONE_CHANGE','AMT_REQ_CREDIT_BUREAU_DAY', 'DAYS_EMPLOYED','REGION_RATING_CLIENT_W_CITY']

In [ ]:
bivar_non_defaulters=non_defaulters_data[bivar_cols]
bivar_non_defaulters_correlation = bivar_non_defaulters.corr()

bivar_non_defaulters_correlation

In [ ]:
# figure size
plt.figure(figsize=(30,20))

# plot
sns.heatmap(bivar_non_defaulters_correlation, annot=True,linewidths=0.10 , annot_kws={"size": 10},cbar=True,square=True)
plt.show()

In [ ]:
new_corr=bivar_non_defaulters_correlation.unstack().reset_index()
new_corr.columns=['Var1','Var2','CORR']
new_corr=new_corr[new_corr['Var1']!=new_corr['Var2']]
new_corr.sort_values('CORR',ascending=False).round(3)

In [ ]:
print(new_corr.sort_values(by='CORR',ascending=False).head(6))
print(new_corr.sort_values(by='CORR',ascending=False).tail(6))

### Top 6 positive correlation
1.  AMT_GOODS_PRICE vs AMT_CREDIT 
2.  REGION_RATING_CLIENT_W_CITY vs  REGION_RATING_CLIENT
3.  CNT_CHILDREN vs CNT_FAM_MEMBERS   0.9
4.  REGION_RATING_CLIENT vs REGION_POPULATION_RELATIVE
5.  AMT_ANNUITY vs AMT_CREDIT
6.  AMT_GOODS_PRICE vs AMT_ANNUITY  

### Bottom 6 negative correlation
6. CNT_CHILDREN         vs         AMT_CREDIT  
5. AMT_REQ_CREDIT_BUREAU_DAY       vs         EXT_SOURCE_2  
4. AMT_REQ_CREDIT_BUREAU_DAY        vs        CNT_CHILDREN  
3. DAYS_EMPLOYED vs  AMT_REQ_CREDIT_BUREAU_DAY  
2. AMT_REQ_CREDIT_BUREAU_DAY       vs      CNT_FAM_MEMBERS   
1. DAYS_EMPLOYED vs  REGION_POPULATION_RELATIVE   

In [ ]:

bivar_defaulters=defaulters_data[bivar_cols]
bivar_defaulters_correlation = bivar_defaulters.corr()

In [ ]:
# figure size
plt.figure(figsize=(30,20))

# plot
sns.heatmap(bivar_defaulters_correlation, annot=True,linewidths=0.10 , annot_kws={"size": 10},cbar=True,square=True)
plt.show()

In [ ]:
new_corrd=bivar_defaulters_correlation.unstack().reset_index()
new_corrd.columns=['Var1','Var2','CORR']
new_corrd=new_corrd[new_corrd['Var1']!=new_corrd['Var2']]
new_corrd.sort_values('CORR',ascending=False).head(20)

In [ ]:
print(new_corr.sort_values(by='CORR',ascending=False).head(6))
print(new_corr.sort_values(by='CORR',ascending=False).tail(6))

### Top 6 positive correlation
1.  AMT_GOODS_PRICE vs AMT_CREDIT 
2.  REGION_RATING_CLIENT_W_CITY vs  REGION_RATING_CLIENT
3.  CNT_CHILDREN vs CNT_FAM_MEMBERS   
4.  AMT_ANNUITY vs AMT_CREDIT
5.  AMT_GOODS_PRICE vs AMT_ANNUITY 
6.  REGION_RATING_CLIENT vs REGION_POPULATION_RELATIVE
 

### Bottom 6 negative correlation
6. CNT_CHILDREN         vs         AMT_CREDIT  
5. AMT_REQ_CREDIT_BUREAU_DAY       vs         EXT_SOURCE_2  
4. AMT_REQ_CREDIT_BUREAU_DAY        vs        CNT_CHILDREN  
3. DAYS_EMPLOYED vs  AMT_REQ_CREDIT_BUREAU_DAY  
2. AMT_REQ_CREDIT_BUREAU_DAY       vs      CNT_FAM_MEMBERS   
1. DAYS_EMPLOYED vs  REGION_POPULATION_RELATIVE   

# Analysis of Previous Application Data

#### 1. Generate new Data Frame and include only ```SK_ID_CURR``` and ```TARGET``` variables

In [ ]:
new_app_data=application_train[['SK_ID_CURR','TARGET']]
new_app_data.shape

#### 2. Generate Data Frame for previous application
##### Read data from file

In [ ]:
previous_app_data=pd.read_csv(path+'/previous_application.csv')

In [ ]:
previous_app_data.shape

In [ ]:
previous_app_data.head()

In [ ]:
pd.set_option('display.max_rows', previous_app_data.shape[0]+1)

#calcualte nulls, percent of nulls,data type,unique,total records, etc
previous_app_data_info=previous_app_data.agg(['count','nunique']).T
previous_app_data_info['nulls']=previous_app_data.isnull().sum()
previous_app_data_info['percent']=(100*previous_app_data.isnull().sum())/len(previous_app_data)
previous_app_data_info['datatype']=previous_app_data.dtypes

In [ ]:
previous_app_data_info

In [ ]:
# Exclude columns with over 25% null
previous_app_data_info=previous_app_data_info[previous_app_data_info['percent']<26]
previous_app_data_info.shape[0]

In [ ]:
list(previous_app_data_info.index)

In [ ]:
previous_app_data=previous_app_data[list(previous_app_data_info.index)]
previous_app_data.head()

**We will not do any further data cleaning and suffice with this**

### Merge both data file to new one using ```SK_ID_CURR``` key

In [ ]:
merged_app_data=pd.merge(new_app_data,previous_app_data,how='inner',on='SK_ID_CURR')

In [ ]:
merged_app_data.shape

# Analysis

## 1. Univariate analysis

In [ ]:
# columns
merged_app_data.columns

**1. HOUR_APPR_PROCESS_START**

In [ ]:
# Which hour did the client application started application
plt.figure(figsize=(8,6))
sns.distplot(merged_app_data['HOUR_APPR_PROCESS_START'], hist=True, kde=False,bins=15, color = 'green', hist_kws={'edgecolor':'black'})
plt.title('Which hours client applied',pad=20,fontsize=14,color='black');

In [ ]:
# separate for TARGET defaulter and non-defaulters
defaulters_data=merged_app_data[merged_app_data.TARGET==1] 
non_defaulters_data=merged_app_data[merged_app_data.TARGET==0]

In [ ]:
custom_histogram('HOUR_APPR_PROCESS_START')

**Insights :** <br/>
Most of the applications are processed during 10AM to 4PM

In [ ]:
#custom function for creating pie plot
def target_univariate_anlysis_pie2_single(x,fontcolor,plot_title,colors=[]):
    plt.figure(figsize=(10,10),dpi=150)
    merged_app_data2=list(merged_app_data[x].value_counts())    
    labels_non_defaulters=list(merged_app_data[x].value_counts().index)
    ax1 = plt.subplot2grid((2,2),(0,0))

    plt.pie(merged_app_data2,labels=labels_non_defaulters,autopct='%1.1f%%', shadow=False,textprops={'fontsize':8,'color':fontcolor},colors=colors)
    plt.title(plot_title ,pad=10,fontsize=10,color='black')
    
    plt.show()

**2. WEEKDAY_APPR_PROCESS_START**

In [ ]:
merged_app_data['WEEKDAY_APPR_PROCESS_START'].unique()

**3. PRODUCT_COMBINATION**

In [ ]:
target_univariate_anlysis_pie2_single('PRODUCT_COMBINATION','black','Weekday daily application process',['lightgreen', 'gold', 'yellow', 'lightblue', 'grey', 'indigo', 'violet'])

**Analys**:<br/>
**Cash**, **POS household with interest**, and **POS mobile with interest** are the top 3 products combination for previous loan applications

In [ ]:
Target_univariate_analysis_pie('WEEKDAY_APPR_PROCESS_START')

**Analysis**<br>
Most of the previsous applications are processed between on Fridays, Mondays, Tuesdays and Wednesdays. While alot of applications are also processed on weekends like Sat and Sun<br>
When separating by defaulter and non-defaulter no significant difference found

In [ ]:
target_univariate_anlysis_pie2_single('NAME_CONTRACT_STATUS','black','Previous application contract status',['magenta','steelblue','orange','olive'])

In [ ]:
Target_univariate_analysis_pie('NAME_CONTRACT_STATUS')

**Analys**:<br/>
**Fewer percentage approved while hiher percetage cancelled comparing to non-defaulters in previous application**  

**4. CHANNEL_TYPE**

# Bivariate analaysis

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.boxplot(x = 'NAME_CONTRACT_TYPE' , y = 'AMT_APPLICATION',hue = 'NAME_CONTRACT_STATUS',
                 data = merged_app_data,palette='rainbow')
# use log 
ax.set_yscale('log')

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.boxplot(x = 'TARGET' , y = 'AMT_APPLICATION',hue = 'NAME_CONTRACT_STATUS',
                 data = merged_app_data,palette='rainbow')
# use log 
ax.set_yscale('log')

**Analysis:**<br/>
**Cash Loans** refusals are more when compared with other loans.<br>
No **Unused offer** reported in **Cash loans** and **Revolving loans**.<br>
There are no **cancellation** reported 

**Analysis**<br>
Here, it shows a positive correlation between AMT_CREDIT and AMT_APPLICATION

In [ ]:
#Channel wise analysis of amout of application
plt.figure(figsize=(15,8))
g1 = sns.barplot(x = 'CHANNEL_TYPE', y = 'AMT_APPLICATION', data = merged_app_data,estimator = np.sum)
plt.title('Channelwise Client',pad=10,fontsize=18,color='black')
plt.xticks(horizontalalignment='right',rotation=45);

**Analysis :** <br/>
Considering the sum of application amount,credit and cash offices had the highest amount in total

In [ ]:
plt.figure(figsize=(15, 8))
sns.barplot(x = 'AMT_CREDIT' , y = 'NAME_CLIENT_TYPE',data = merged_app_data)
plt.title("Credit amount distribution for client type",pad=10,fontsize=18,color='black');

**Analysis :**<br> 
Alot of people have difficulties to get loans due to insufficient credit histories. From above it is seen that old clients are the more favored above others to provide loans by the loan comapnies.<br/>
Loans are also given to customers whose data is not available with the Loan companies(Client Type=XNA). This could be a FRAUD or some error in capturing client information through th system. It is necessary that loan companies capture full and complete information for each client.

## Finding correlation

In [ ]:
# usign scatter plot to show relationship of AMT CREDIT and AMT Application
plt.figure(figsize=(15, 8))
sns.scatterplot(x=merged_app_data['AMT_CREDIT'], y=merged_app_data['AMT_APPLICATION'])
plt.title("Relationship of amout of Credit and Application")
plt.show()

**columns to find correlation between them**

In [ ]:
merged_app_data.select_dtypes('int64').columns

In [ ]:
merged_app_data.select_dtypes('float64').columns

In [ ]:
cols=['WEEKDAY_APPR_PROCESS_START','AMT_APPLICATION','AMT_CREDIT','NFLAG_LAST_APPL_IN_DAY',
      'DAYS_DECISION', 'SELLERPLACE_AREA','AMT_ANNUITY','AMT_GOODS_PRICE','CNT_PAYMENT']

In [ ]:
merged_app_data=merged_app_data[cols]
merged_app_data_corr=merged_app_data.corr()

In [ ]:
# figure size
plt.figure(figsize=(30,20))

# plot
sns.heatmap(merged_app_data_corr, annot=True,linewidths=0.10 , annot_kws={"size": 10},cbar=True,square=True)
plt.show()

In [ ]:
new_cormat=merged_app_data_corr
corr=new_cormat.where(np.triu(np.ones(new_cormat.shape),k=1).astype(np.bool))
corr=corr.abs().unstack().reset_index()
corr.columns=['Var1','Var2','CORR']
corr.dropna(subset=['CORR'],inplace=True)
corr.sort_values('CORR',ascending=False).head(20)
# new_corrdp=new_corrd[new_corrdp['Var1']!=new_corrdp['Var2']]


**Analysis:**<br>
**Following is top 4 and bottom 4 correlated variables**

In [ ]:
print(new_corr.sort_values(by='CORR',ascending=False).head(6))
print(new_corr.sort_values(by='CORR',ascending=False).tail(6))

Loan Amount requested by client orignally and final credit loan ammount on the previous application given to the client are highly correlated as seen from above scatter and heat maps. This means the loan companies are providing almost closest amount of for which the client has made an initial request for. 
<br/>We have already seen earlier that consumer loans are the more offered than other loans 

# THE END